In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import math
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
# 104x1 + 128x2 + 135x3 + 139x4 + 150x5 + 153x6 + 162x7 + 168x8 + 195x9 + 198x10
objective_coeff = [104, 128, 135, 139, 150, 153, 162, 168, 195, 198]
# 9x1^2 + 8x2^2 + 7x3^2 + 7x4^2 + 6x5^2 + 6x6^2 + 5x7^2 + 2x8^2 + x9^2 + x10^2 <= 68644
constraint_coeff = [9, 8, 7, 7, 6, 6, 5, 2, 1, 1]
constraint_bound = 68644
variable_upper_bound = math.sqrt(constraint_bound)

In [10]:
# Calculate the real value solution for the given function f(x) for all the vaiables
def real_value_calculation(objective_coeff_array, constraint_coeff_array, constraint_bound):
    if len(objective_coeff_array) == 0:
        return []

    ans = []
    if constraint_bound <= 0:
        for i in range(len(objective_coeff_array)):
            ans.append(0)
        return ans

    total_sum = 0
    for i in range(len(objective_coeff_array)):
        total_sum += constraint_coeff_array[i] * math.pow((objective_coeff_array[i] / (2 * constraint_coeff_array[i])), 2)

    exr = math.sqrt(total_sum / constraint_bound)
    for i in range(len(objective_coeff_array)):
        ans.append(objective_coeff_array[i] / (2 * exr * constraint_coeff_array[i]))

    return ans

In [11]:
# Get the maximum value once the all the values have been assigned
def get_max_value(objective_coeff_array, solution_val, floor_true=False):
    total_sum = 0
    for i in range(len(solution_val)):
        if floor_true:
            total_sum += objective_coeff_array[i] * math.floor(solution_val[i])
        else:
            total_sum += objective_coeff_array[i] * solution_val[i]
    return total_sum

In [12]:
# Calculate the integer value solution for the given function f(x) for all the variables
def integer_value_calculation(lower_bound, upper_bound, index, variable_size, assign_variables, output_max, bounds):
    print(assign_variables, bounds)

    if index == -1:
        max_value = get_max_value(assign_variables, objective_coeff)
        output_max[2] += 1 # Update the assignment values 
        if max_value > output_max[0]:
            output_max[0] = max_value
            output_max[1] = assign_variables.copy()
            bounds[0] = max_value
            print("Maximum Value obtained from the assignment", output_max[0])
        return True

    greedy_assignment_value = variable_upper_bound
    new_constraint = constraint_bound
    lower_bound = bounds[0]
    upper_bound = bounds[1]

    for i in range(index + 1, variable_size):
        new_constraint -= math.pow(assign_variables[i], 2) * constraint_coeff[i]

    while greedy_assignment_value >= 0:
        constraint = new_constraint
        constraint -= math.pow(greedy_assignment_value, 2) * constraint_coeff[index]

        if constraint < 0:
            greedy_assignment_value -= 1
            continue

        temp_solution = real_value_calculation(objective_coeff[:index], constraint_coeff[:index], constraint)
        temp_solution.append(greedy_assignment_value)
        temp_solution.extend(assign_variables[index + 1:])

        temp_max = get_max_value(temp_solution, objective_coeff, False)

        if lower_bound <= temp_max <= upper_bound:
            break
        greedy_assignment_value -= 1

    variable_upper_bound_new = greedy_assignment_value
    if variable_upper_bound_new == -1:
        return

    greedy_assignment_value = 0
    while greedy_assignment_value <= variable_upper_bound:
        constraint = new_constraint
        constraint -= math.pow(greedy_assignment_value, 2) * constraint_coeff[index]

        if constraint < 0:
            greedy_assignment_value += 1
            continue

        temp_solution = real_value_calculation(objective_coeff[:index], constraint_coeff[:index], constraint)
        temp_solution.append(greedy_assignment_value)
        temp_solution.extend(assign_variables[index + 1:])

        temp_max = get_max_value(temp_solution, objective_coeff, False)

        if lower_bound <= temp_max <= upper_bound:
            break

        greedy_assignment_value += 1

    variable_lower_bound = greedy_assignment_value
    if variable_lower_bound == variable_upper_bound + 1:
        return

    for assignment in range(int(variable_upper_bound_new), int(variable_lower_bound) - 1, -1):
        constraint = new_constraint
        constraint -= math.pow(assignment, 2) * constraint_coeff[index]

        temp_solution = real_value_calculation(objective_coeff[:index], constraint_coeff[:index], constraint)
        temp_solution.append(assignment)
        temp_solution.extend(assign_variables[index + 1:])

        function_upper_bound = get_max_value(temp_solution, objective_coeff, False)
        function_lower_bound = max(lower_bound, get_max_value(temp_solution, objective_coeff, True))

        assign_variables[index] = assignment
        is_solvable = integer_value_calculation(function_lower_bound, function_upper_bound, index - 1, variable_size,
                                         assign_variables, output_max, bounds)


In [13]:
# Get the original real value solution for f(x)
original_var_solution = real_value_calculation(objective_coeff, constraint_coeff, constraint_bound)
global_max = 0
global_min = 0

for i in range(len(original_var_solution)):
    global_max += original_var_solution[i] * objective_coeff[i]
    global_min += math.floor(original_var_solution[i]) * objective_coeff[i]

bounds = [global_min, global_max]

index = 9
assign_variables = [-1] * len(objective_coeff)
output_max = [-1, [], 0] # Max value, Assignement values, Total no of assignment

In [14]:
integer_value_calculation(global_min, global_max, 9, 10, assign_variables, output_max, bounds)
print(output_max)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1] [87441, 88015.93234412931]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, -1, -1, -1, -1, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, -1, -1, -1, 58, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, -1, -1, 22, 58, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, -1, 18, 22, 58, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 18, 22, 58, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 22, 58, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 22, 57, 143, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 22, 57, 142, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 22, 60, 142, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 23, 60, 142, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 23, 60, 142, 177] [87441, 88015.93234412931]
[-1, -1, -1, -1, 17, 17, 23, 60, 142, 177] [87441, 88015.93234412931]
[-1, -1, -1, 13, 17, 17